[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/bigdata_analyst_cert_v2/blob/main/part4/ch9/p9_type3.ipynb)

## 작업형3

### 문제1-1

In [ ]:
import pandas as pd
df = pd.read_csv("design.csv")
# df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert_v2/main/part4/ch9/design.csv")
df.head()

,id,design,c1,c2,c3,c4,c5
0,1,58.962471,0.374540,0.642032,0.103124,0.168935,0.707239
1,2,42.006534,0.950714,0.084140,0.902553,0.278590,0.152539
2,3,55.831980,0.731994,0.161629,0.505252,0.177010,0.576288
3,4,64.458592,0.598658,0.898554,0.826457,0.088703,0.606715
4,5,61.342792,0.156019,0.606429,0.320050,0.120636,0.424131


In [ ]:
from statsmodels.formula.api import ols

# 1) train, test 데이터 분리
cond1 = df['id'] <= 140
cond2 = df['id'] > 140
train = df[cond1].copy()
test = df[cond2].copy()

# 2) 전체 변수 사용 회귀분석
model = ols("design ~ c1 + c2 + c3 + c4 + c5", data=train).fit()
print(model.summary())

# 3) 상수항(Intercept)은 제외하고, p-value가 0.05보다 작은 독립변수의 개수를 계산
print(model.pvalues[1:] < 0.05)
sum(model.pvalues[1:] < 0.05)

                            OLS Regression Results                            
Dep. Variable:                 design   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     9.697
Date:                Mon, 03 Mar 2025   Prob (F-statistic):           6.37e-08
Time:                        11:59:34   Log-Likelihood:                -468.72
No. Observations:                 140   AIC:                             949.4
Df Residuals:                     134   BIC:                             967.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     53.0220      2.294     23.112      0.0

3

### 문제1-2

In [ ]:
# 방법1
# 1) 유의한 변수만 사용한 회귀 분석
model = ols("design ~ c1 + c2 + c4", data=train).fit()

# 2) train 데이터에서 design의 예측
train['pred_design'] = model.predict(train)

# 3) 피어슨 상관계수 계산
result = train['design'].corr(train['pred_design'])
round(result,3)

0.501

In [ ]:
# 방법2(추가tip)
from scipy.stats import pearsonr
# 1) 유의한 변수만 사용한 회귀 분석
model = ols("design ~ c1 + c2 + c4", data=train).fit()

# 학습데이터에서 예측값 계산
train['pred_design'] = model.predict(train)

# 예측값과 실제 design 값 사이의 피어슨 상관계수 계산
c, p = pearsonr(train['design'], train['pred_design'])
round(c,3)

0.501

### 문제1-3

In [ ]:
# 1) test데이터에서 design값 계산
test['pred_design'] = model.predict(test)

# 2) test데이터에 대한 RMSE 계산
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(test['design'], test['pred_design'])
round(rmse, 3)

8.488

### 문제2-1

In [ ]:
import pandas as pd
df = pd.read_csv("retention.csv")
# df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert_v2/main/part4/ch9/retention.csv")
df.head()

,CustomerID,MonthlyCharges,CustomerTenure,HasPhoneService,HasTechInsurance,Churn
0,1,77.450712,47,1,1,1
1,2,67.926035,68,1,1,0
2,3,79.715328,45,0,1,1
3,4,92.845448,2,1,0,1
4,5,66.487699,27,1,1,0


In [ ]:
import pandas as pd
from statsmodels.formula.api import logit

# 1) 로지스틱 회귀분석 수행
formula = 'Churn ~ MonthlyCharges + CustomerTenure + HasPhoneService + HasTechInsurance'
model = logit(formula, data=df).fit()

# 2) 학습된 모델의 회귀 분석 결과 출력
print(model.summary())

# 3) 'MonthlyCharges' 변수의 p-value 확인
round(model.pvalues['MonthlyCharges'], 3)

Optimization terminated successfully.
         Current function value: 0.582234
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Churn   No. Observations:                   80
Model:                          Logit   Df Residuals:                       75
Method:                           MLE   Df Model:                            4
Date:                Mon, 03 Mar 2025   Pseudo R-squ.:                  0.1585
Time:                        12:06:06   Log-Likelihood:                -46.579
converged:                       True   LL-Null:                       -55.352
Covariance Type:            nonrobust   LLR p-value:                  0.001513
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -4.4731      1.437     -3.114      0.002      -7.289      -1.657
MonthlyChar

0.008

### 문제2-2

In [ ]:
# 1) 오즈비 계산
import numpy as np
result = np.exp(model.params['HasPhoneService'])
round(result, 3)

0.701

### 문제2-3

In [ ]:
# 1) 각 고객의 이탈 확률을 예측
pred_probs = model.predict(df)

# 2) 예측한 이탈 확률이 0.3 초과하는 고객의 수
sum(pred_probs > 0.3)

65